# Paper Analyses

Replication of the analyses in the paper: TBC

In [1]:
import os
import time

from deepmet.workflows import train_single_model
from deepmet.auxiliary import get_fingerprints_from_meta, select_features, Config
from deepmet.datasets import load_training_dataset

Compounds were extracted from the HMDB and ZINC12 databases, subject to the following constraints:
- Exact mass filter: 100Da < exact mass \> 800Da
- Other things

The entire set of compounds passing these filters in HMDB were retained while a random sample of 20,000 compounds were taken from ZINC12. The smiles for these compounds are available in the `data/test_set` folder.

In [2]:
from shutil import copytree
from tests.auxiliary import get_normal_non_normal_subsets

# Path to write results
results_path = os.path.join(os.path.dirname(os.path.dirname(os.path.realpath(__file__))), "paper_results")
os.mkdir(results_path)

# Copy input data to the results folder
copytree(
    os.path.join(os.path.dirname(os.path.dirname(os.path.realpath(__file__))), "deepmet", "data"),
    os.path.join(results_path, "data")
)

# Location of the "normal" and "non-normal" smiles
normal_meta_path, non_normal_meta_path = get_normal_non_normal_subsets(results_path)

if not os.path.exists(results_path):
    os.mkdir(results_path)

The function `train_likeness_scorer` implements the workflow for training the DeepMet model.
For the purposes of this vignette, the individual steps will be carried out manually.

While smiles are provided in the data files, these are not used as input to the model.
If not given to the `train_likeness_scorer` function, these will be converted to molecular
fingerprints using the smiles given as input. These are calculated in the following chunk.

This is a particularly time-consuming step, so it is recommended not to unnecessarily regenerate the fingerprints.

In [3]:
processed_normal_fingerprints_path = os.path.join(results_path, "normal_fingerprints.csv")
processed_non_normal_fingerprints_path = os.path.join(results_path, "non_normal_fingerprints.csv")

normal_fingerprints_path = get_fingerprints_from_meta(normal_meta_path, processed_normal_fingerprints_path)
non_normal_fingerprints_path = get_fingerprints_from_meta(non_normal_meta_path, processed_non_normal_fingerprints_path)

Here, we set the seed and set the training options for training DeepMet. The learning rate was
selected that was associated with the minimum loss on the validation set.

In [4]:
# Seed to be used for loading the dataset and training models
seed = 1

# Settings required by the DeepMet model
cfg = Config({
    "net_name": "cocrystal_transformer",
    "objective": "one-class",
    "nu": 0.1,
    "rep_dim": 200,
    "seed": seed,
    "optimizer_name": "amsgrad",
    "lr": 0.000155986,
    "n_epochs": 20,
    "lr_milestones": tuple(),
    "batch_size": 2000,
    "weight_decay": 1e-5,
    "pretrain": False,
    "in_features": 2800,
    "device": "cpu"
})

While we have now generated the molecular fingerprints, these include many poorly balanced and
redundant features. We therefore use `select_features` to remove redundant and unbalanced features
prior to model training.

The data is then loaded into a torch-compatible format using `load_training_dataset`.

In [5]:
normal_fingerprints_path, non_normal_fingerprints_paths, selected_features = select_features(
        normal_fingerprints_path=normal_fingerprints_path,
        normal_fingerprints_out_path=os.path.join(results_path, "selected_normal_fingerprints.csv"),
        non_normal_fingerprints_paths=non_normal_fingerprints_path,
        non_normal_fingerprints_out_paths=os.path.join(results_path, "selected_non_normal_fingerprints.csv")
)

cfg.settings["selected_features"] = selected_features

# select_features allows for the simultaneous selection of multiple non-normal datasets
# we only have a single non-normal ZINC12 set here, which we will use to evaluate the final model
non_normal_fingerprints_path = non_normal_fingerprints_paths[0]

dataset, dataset_labels, validation_dataset = load_training_dataset(
    normal_dataset_path=normal_fingerprints_path,
    normal_meta_path=normal_meta_path,
    non_normal_dataset_path=non_normal_fingerprints_path,
    non_normal_dataset_meta_path=non_normal_meta_path,
    seed=seed,
    validation_split=0.8,
    test_split=0.9
)

With the dataset loaded, we can now train the model. The core training workflow
is carried out using `train_single_model`. With the selected parameters, the final
AUC on the test set is 97.91% - importantly, AUC was not used for hyperparameter
optimisation as the validation set did not contain any "non-normal" compounds.

In [6]:
# Train the model (loss is calculated on the 'normal' validation set for parameter tuning)
deep_met_start = time.time()
deep_met_model = train_single_model(cfg, validation_dataset)
deep_met_end = time.time()

# Test using separate test dataset (includes the ZINC12 set of 'non-normal' compounds)
deep_met_model.test(dataset, device="cpu")

print("AUC on test set: " + str(round(deep_met_model.results["test_auc"], 4)))

C:\Users\jackg\Miniconda3\envs\DeepMet\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Only one class present in y_true. ROC AUC score is not defined in that case.
AUC on test set: 0.98


With DeepMet trained, we can train isolation forest and one-class SVM models for comparison. As for the DeepMet
model, non-normal compounds are not used for parameter selection. The contamination and nu parameters were set to 0.1
for consistency with DeepMet. The remaining isolation forest parameters and the OC-SVM kernel are the same
as were used for the Co-crystal paper. The gamma parameter was selected using the validation set and the scaled distance of the outliers
to the hyperplane as a loss function.

In [7]:
import pickle
from pyod.models import ocsvm, iforest

iforest_model = iforest.IForest(
    contamination=0.1,
    n_estimators=400,
    behaviour="new",
    random_state=seed,
    max_samples=1000
)

ocsvm_model = ocsvm.OCSVM(
    contamination=0.1,
    kernel="rbf",
    nu=0.1,
    gamma=0.00386
)

x_train = validation_dataset.train_set.dataset.data[validation_dataset.train_set.indices]

iforest_start = time.time()
iforest_model.fit(x_train)
iforest_end = time.time()

ocsvm_start = time.time()
ocsvm_model.fit(x_train)
ocsvm_end = time.time()

pickle.dump(iforest_model, open(os.path.join(results_path, "iforest_model.pkl"), "wb"))
pickle.dump(ocsvm_model, open(os.path.join(results_path, "ocsvm_model.pkl"), "wb"))

The isolation forests and OC-SVM models take a long time to train relative to DeepMet.

In [8]:
print("DeepMet training time: " + str(round(deep_met_end - deep_met_start)) + " seconds")
print("Isolation forests training time: " + str(round(iforest_end - iforest_start)) + " seconds")
print("OC-SVM training time: " + str(round(ocsvm_end - ocsvm_start)) + " seconds")

DeepMet training time: 164 seconds
Isolation forests training time: 691 seconds
OC-SVM training time: 434 seconds


We can calculate AUC for these models as was done for DeepMet. Both the isolation forests and the OC-SVM
models have similar discriminative performance; they both have a lower AUC compared to DeepMet.

In [9]:
from sklearn.metrics import roc_auc_score

x_test = dataset.test_set.dataset.data[dataset.test_set.indices]
labels_test = dataset.test_set.dataset.labels[dataset.test_set.indices]

print("Isolation forest AUC: " + str(round(roc_auc_score(labels_test, iforest_model.decision_function(x_test)), 2)))
print("OC-SVM AUC: " + str(round(roc_auc_score(labels_test, ocsvm_model.decision_function(x_test)), 2)))

Isolation forest AUC: 0.94
OC-SVM AUC: 0.89
